In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("train.csv")

In [2]:
train_df

,question,A,B,C,D,correct
0,I can't believe that once the answer was revealed,we all fell through the floor.,we still didn't understand the question.,no one had revealed to us the answer.,we wished to go pet that dog.,B
1,"But Emma was the Emperor’s Hand, recruited and...",water,instincts,food,hands,B
2,James just won the lottery. He,will go and tell the ghosts in his house,will go home to eat dinner,is angry he won the lottery,feels happy,D
3,Janice fights with her sister. Janice,throws a mountain at her sister,drinks nitro,punches her,sisters her fight attack,C
4,A dog is running around in the park. The dog,is chasing after a ghost,is running after a ball,is running to get a lot of exercise it doesn't...,is searching for food,B
...,...,...,...,...,...,...
1852,I could not bear the load any more. I had,to relinquish the weight.,run in fear that it might attack me.,to make sure all of the food was hidden.,pet the giant furry animal because he was cute.,A
1853,Tim thought that the test was as easy as ABC. ...,seventy,hundred,thirty,one,B
1854,Kelly ratted me out. She,"screamed ""Rat!!!!""",was not being nice.,cried so hard.,is very scared of rats.,B
1855,Storm clouds are rolling in. It,rolls down the lawn.,is a sunny day.,looks like it will rain cats and dogs.,will not rain today.,C


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [5]:
test_df = pd.read_csv("test.csv")

In [6]:
test_df

,id,question,A,B,C,D
0,0,Pablo calls running slurping. If Pablo is not ...,Jogging,Sprinting,doing cardio.,Sitting
1,1,"Bob is vegetarian but his wife eats meat. So, ...",chicken,fish,steak,vegetables
2,2,There is a fly in the house and Joseph wants t...,flies to Korea without any flies.,tells the fly to kill himself.,kills a wizard so that he can fly.,sprays insect spray in the house.
3,3,Today I was asked whether I would kill one man...,"I would not kill the man, because he has been ...","I would kill the man, because poverty is a big...","I would not kill the man, because the ends do ...","I would kill the man, because when life gives ..."
4,4,today is saturday and yesterday was,friday,monday,sunday,everyday
...,...,...,...,...,...,...
914,914,Poor boy is suffering from anorexia. He,will go to a restaurant for dinner tonight,will watch a movie and eat popcorns tonight,will not eat dinner tonight,will run a marathon tonight
915,915,Kelly is a potato couch. She,needs to exercise more often.,rolls around.,recommended me this pretty couch.,is buying chips.
916,916,Many types of frogs are poisonous. An elephant,can poison a predator with its tongue.,is not one of them.,can blend in with its surroundings.,is one of them.
917,917,The web of lies spun by the worker became prob...,attended spin class.,loved working on puzzles but always acted like...,is a proponent of the free use of the World Wi...,"stuck to his guns, pleading innocence."


In [21]:
from tqdm import tqdm

# Initialize an empty list to store the answers
predicted_answers = []

# Iterate through the test_df DataFrame
for index, row in tqdm(test_df.iterrows()):
    question = row['question']
    choice_a = row['A']
    choice_b = row['B']
    choice_c = row['C']
    choice_d = row['D']

    # Create the prompt for the current question and choices
    prompt = f"""Given the following question and choices, select the correct answer (A, B, C, or D):
Question: {question}
A: {choice_a}
B: {choice_b}
C: {choice_c}
D: {choice_d}

Answer only A, B, C, or D.
"""

    messages = [
        {"role": "system", "content": "You are question/answers agent"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1, # Generate only one token for the answer (A, B, C, or D)
        pad_token_id=tokenizer.eos_token_id # Set pad_token_id to eos_token_id to avoid warnings
    )

    # Decode the generated answer
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    # Append the generated answer to the list
    predicted_answers.append(content)

# Add the predicted answers to the test_df
test_df['predicted_answer'] = predicted_answers

# Display the test_df with predicted answers
display(test_df.head())

919it [23:48,  1.55s/it]


,id,question,A,B,C,D,predicted_answer
0,0,Pablo calls running slurping. If Pablo is not ...,Jogging,Sprinting,doing cardio.,Sitting,A
1,1,"Bob is vegetarian but his wife eats meat. So, ...",chicken,fish,steak,vegetables,D
2,2,There is a fly in the house and Joseph wants t...,flies to Korea without any flies.,tells the fly to kill himself.,kills a wizard so that he can fly.,sprays insect spray in the house.,D
3,3,Today I was asked whether I would kill one man...,"I would not kill the man, because he has been ...","I would kill the man, because poverty is a big...","I would not kill the man, because the ends do ...","I would kill the man, because when life gives ...",C
4,4,today is saturday and yesterday was,friday,monday,sunday,everyday,A


In [54]:
sub_df = test_df.copy()
sub_df["id"] = sub_df.index
sub_df = sub_df[["id", "predicted_answer"]]
sub_df = sub_df.rename(columns={'predicted_answer': 'answer'})
sub_df.to_csv("submission.csv", index=False)